run this cell block if dataset isn't local in your drive

if so, consider running next cell

In [ ]:
# data moved to drive. ignore redownload
!wget http://158.109.8.102/AppaRealAge/appa-real-release.zip
!mkdir data_dir 
!unzip appa-real-release.zip -d data_dir

Streaming output truncated to the last 5000 lines.
  inflating: data_dir/appa-real-release/test/007580.jpg  
  inflating: data_dir/appa-real-release/test/007263.jpg  
  inflating: data_dir/appa-real-release/test/006471.jpg  
  inflating: data_dir/appa-real-release/test/005662.jpg.mat  
  inflating: data_dir/appa-real-release/test/005781.jpg  
  inflating: data_dir/appa-real-release/test/006207.jpg.mat  
  inflating: data_dir/appa-real-release/test/006199.jpg  
  inflating: data_dir/appa-real-release/test/006799.jpg  
  inflating: data_dir/appa-real-release/test/006339.jpg.mat  
  inflating: data_dir/appa-real-release/test/006214.jpg  
  inflating: data_dir/appa-real-release/test/006340.jpg_face.jpg  
  inflating: data_dir/appa-real-release/test/006447.jpg.mat  
  inflating: data_dir/appa-real-release/test/007244.jpg  
  inflating: data_dir/appa-real-release/test/007017.jpg_face.jpg  
  inflating: data_dir/appa-real-release/test/006012.jpg  
  inflating: data_dir/appa-real-release/test/

# load drive
dataset and recent checkpoint are stored on google drive.

access to avoid re-download

In [ ]:
# start
# mount drive to access data and recent saved model          
from google.colab import drive                              
drive.mount('/content/drive')                               

Mounted at /content/drive


make dataset local in colab if available in your drive

skip if you executed the first cell block


In [ ]:
!cp /content/drive/MyDrive/age_esti_colab_session/appa-real-release.zip ./
!mkdir data_dir
!unzip appa-real-release.zip  -d data_dir
!rm appa-real-release.zip

Streaming output truncated to the last 5000 lines.
  inflating: data_dir/appa-real-release/test/007580.jpg  
  inflating: data_dir/appa-real-release/test/007263.jpg  
  inflating: data_dir/appa-real-release/test/006471.jpg  
  inflating: data_dir/appa-real-release/test/005662.jpg.mat  
  inflating: data_dir/appa-real-release/test/005781.jpg  
  inflating: data_dir/appa-real-release/test/006207.jpg.mat  
  inflating: data_dir/appa-real-release/test/006199.jpg  
  inflating: data_dir/appa-real-release/test/006799.jpg  
  inflating: data_dir/appa-real-release/test/006339.jpg.mat  
  inflating: data_dir/appa-real-release/test/006214.jpg  
  inflating: data_dir/appa-real-release/test/006340.jpg_face.jpg  
  inflating: data_dir/appa-real-release/test/006447.jpg.mat  
  inflating: data_dir/appa-real-release/test/007244.jpg  
  inflating: data_dir/appa-real-release/test/007017.jpg_face.jpg  
  inflating: data_dir/appa-real-release/test/006012.jpg  
  inflating: data_dir/appa-real-release/test/

# install deps

In [ ]:
!pip install pretrainedmodels
!pip install torchsummary
!pip install yacs
!pip install better_exceptions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 4.0 MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=b21414e9f55034e82e821dab6e53430dc161c57cd8de5741c6308e32a6b53dc5
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


import model using pretrainedmodels module.

using transfer learning to train on our dataset

# config file

In [ ]:
%%writefile defaults.py

from yacs.config import CfgNode as CN

_C = CN()

# Model
_C.MODEL = CN()
# _C.MODEL.ARCH = "se_resnext50_32x4d"  # check python train.py -h for available models
_C.MODEL.ARCH = "se_resnext101_32x4d"
_C.MODEL.IMG_SIZE = 224

# Train
_C.TRAIN = CN()
_C.TRAIN.OPT = "adam"  # adam or sgd
_C.TRAIN.WORKERS = 2   # 8
_C.TRAIN.LR = 0.01 # 0.001
_C.TRAIN.LR_DECAY_STEP = 20
_C.TRAIN.LR_DECAY_RATE = 0.2
_C.TRAIN.MOMENTUM = 0.9
_C.TRAIN.WEIGHT_DECAY = 0.0
_C.TRAIN.BATCH_SIZE = 32 #  64 128
_C.TRAIN.EPOCHS = 80
_C.TRAIN.AGE_STDDEV = 1.0

# Test
_C.TEST = CN()
_C.TEST.WORKERS = 8
_C.TEST.BATCH_SIZE = 128


Writing defaults.py


In [ ]:
%%writefile ignore_list.csv
img_name
000025.jpg
000049.jpg
000067.jpg
000085.jpg
000095.jpg
000100.jpg
000127.jpg
000145.jpg
000191.jpg
000215.jpg
000320.jpg
000373.jpg
000392.jpg
000407.jpg
000488.jpg
000503.jpg
000506.jpg
000510.jpg
000536.jpg
000605.jpg
000625.jpg
000639.jpg
000707.jpg
000708.jpg
000712.jpg
000813.jpg
000837.jpg
000848.jpg
000856.jpg
000891.jpg
000892.jpg
001022.jpg
001044.jpg
001095.jpg
001098.jpg
001122.jpg
001125.jpg
001137.jpg
001156.jpg
001227.jpg
001251.jpg
001267.jpg
001282.jpg
001328.jpg
001349.jpg
001380.jpg
001427.jpg
001460.jpg
001475.jpg
001697.jpg
001744.jpg
001864.jpg
001957.jpg
001968.jpg
001973.jpg
002029.jpg
002063.jpg
002109.jpg
002112.jpg
002115.jpg
002123.jpg
002162.jpg
002175.jpg
002179.jpg
002221.jpg
002250.jpg
002303.jpg
002359.jpg
002360.jpg
002412.jpg
002417.jpg
002435.jpg
002460.jpg
002466.jpg
002472.jpg
002488.jpg
002535.jpg
002543.jpg
002565.jpg
002615.jpg
002630.jpg
002633.jpg
002661.jpg
002733.jpg
002756.jpg
002860.jpg
002883.jpg
002887.jpg
002890.jpg
002948.jpg
002995.jpg
003018.jpg
003130.jpg
003164.jpg
003233.jpg
003258.jpg
003271.jpg
003329.jpg
003351.jpg
003357.jpg
003371.jpg
003415.jpg
003427.jpg
003441.jpg
003447.jpg
003458.jpg
003570.jpg
003625.jpg
003669.jpg
003711.jpg
003747.jpg
003749.jpg
003758.jpg
003763.jpg
003772.jpg
003805.jpg
003814.jpg
003903.jpg


Writing ignore_list.csv


# dataset

In [ ]:
%%writefile dataset.py

import argparse
import better_exceptions
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import cv2
from torch.utils.data import Dataset
from imgaug import augmenters as iaa


class ImgAugTransform:
    def __init__(self):
        self.aug = iaa.Sequential([
            iaa.OneOf([
                iaa.Sometimes(0.25, iaa.AdditiveGaussianNoise(scale=0.1 * 255)),
                iaa.Sometimes(0.25, iaa.GaussianBlur(sigma=(0, 3.0)))
                ]),
            iaa.Affine(
                rotate=(-20, 20), mode="edge",
                scale={"x": (0.95, 1.05), "y": (0.95, 1.05)},
                translate_percent={"x": (-0.05, 0.05), "y": (-0.05, 0.05)}
            ),
            iaa.AddToHueAndSaturation(value=(-10, 10), per_channel=True),
            iaa.GammaContrast((0.3, 2)),
            iaa.Fliplr(0.5),
        ])

    def __call__(self, img):
        img = np.array(img)
        img = self.aug.augment_image(img)
        return img


class FaceDataset(Dataset):
    def __init__(self, data_dir, data_type, img_size=224, augment=False, age_stddev=1.0):
        assert(data_type in ("train", "valid", "test"))
        csv_path = Path(data_dir).joinpath(f"gt_avg_{data_type}.csv")
        img_dir = Path(data_dir).joinpath(data_type)
        self.img_size = img_size
        self.augment = augment
        self.age_stddev = age_stddev

        if augment:
            self.transform = ImgAugTransform()
        else:
            self.transform = lambda i: i

        self.x = []
        self.y = []
        self.std = []
        df = pd.read_csv(str(csv_path))
        ignore_path = Path(__file__).resolve().parent.joinpath("ignore_list.csv")
        ignore_img_names = list(pd.read_csv(str(ignore_path))["img_name"].values)

        for _, row in df.iterrows():
            img_name = row["file_name"]

            if img_name in ignore_img_names:
                continue

            img_path = img_dir.joinpath(img_name + "_face.jpg")
            assert(img_path.is_file())
            self.x.append(str(img_path))
            self.y.append(row["apparent_age_avg"])
            self.std.append(row["apparent_age_std"])

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        img_path = self.x[idx]
        age = self.y[idx]

        if self.augment:
            age += np.random.randn() * self.std[idx] * self.age_stddev

        img = cv2.imread(str(img_path), 1)
        img = cv2.resize(img, (self.img_size, self.img_size))
        img = self.transform(img).astype(np.float32)
        return torch.from_numpy(np.transpose(img, (2, 0, 1))), np.clip(round(age), 0, 100)


def main():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument("--data_dir", type=str, required=True)
    args = parser.parse_args()
    dataset = FaceDataset(args.data_dir, "train")
    print("train dataset len: {}".format(len(dataset)))
    dataset = FaceDataset(args.data_dir, "valid")
    print("valid dataset len: {}".format(len(dataset)))
    dataset = FaceDataset(args.data_dir, "test")
    print("test dataset len: {}".format(len(dataset)))


if __name__ == '__main__':
    main()


Writing dataset.py


In [ ]:
!python dataset.py --data_dir ./data_dir/appa-real-release/

train dataset len: 3995
valid dataset len: 1500
test dataset len: 1978


# model

In [ ]:
%%writefile model.py
import torch
import torch.nn as nn
import pretrainedmodels
import pretrainedmodels.utils

from torchsummary import summary

import ssl


model_name_ = "se_resnext101_32x4d"

ssl._create_default_https_context = ssl._create_unverified_context



class ResCusNet(nn.Module):

    def __init__(self):
        super(ResCusNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.maxPool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=3)

        self.conv2 = nn.Conv2d(128, 64, kernel_size=(5, 5))
        self.relu2 = nn.ReLU()
        self.maxPool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.bn1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxPool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxPool2(x)
        
        x = self.bn1(x)
        x = self.relu3(x)
        return x


def get_model(model_name, num_classes=101, pretrained="imagenet"):
    model = pretrainedmodels.__dict__[model_name](pretrained=pretrained)

    for param in model.parameters():
        param.requires_grad = False

    # model.layer0 = ResCusNet()
    model.last_linear = nn.Linear(model.last_linear.in_features, num_classes)
    model.avg_pool = nn.AdaptiveAvgPool2d(1)

    return model


def main():
    model = get_model(model_name_)
    # print(summary(model, (244, 244, 3)))
    print(model)


if __name__ == '__main__':
    main()


Writing model.py


In [ ]:
# model summary
!python model.py

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /root/.cache/torch/hub/checkpoints/se_resnext101_32x4d-3b2fe3d8.pth
100% 187M/187M [09:12<00:00, 356kB/s]
SENet(
  (layer0): ResCusNet(
    (conv1): Conv2d(3, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (relu1): ReLU()
    (maxPool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=3, ceil_mode=False)
    (conv2): Conv2d(128, 64, kernel_size=(5, 5), stride=(1, 1))
    (relu2): ReLU()
    (maxPool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU()
  )
  (layer1): Sequential(
    (0): SEResNeXtBottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size

# train

In [ ]:
%%writefile train.py

import argparse
import better_exceptions
from pathlib import Path
from collections import OrderedDict
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
from torch.optim.lr_scheduler import StepLR
import torch.utils.data
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import pretrainedmodels
import pretrainedmodels.utils
from model import get_model
from dataset import FaceDataset
from defaults import _C as cfg


def get_args():
    model_names = sorted(name for name in pretrainedmodels.__dict__
                         if not name.startswith("__")
                         and name.islower()
                         and callable(pretrainedmodels.__dict__[name]))
    parser = argparse.ArgumentParser(description=f"available models: {model_names}",
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument("--data_dir", type=str, required=True, help="Data root directory")
    parser.add_argument("--resume", type=str, default=None, help="Resume from checkpoint if any")
    parser.add_argument("--checkpoint", type=str, default="checkpoint", help="Checkpoint directory")
    parser.add_argument("--tensorboard", type=str, default=None, help="Tensorboard log directory")
    parser.add_argument('--multi_gpu', action="store_true", help="Use multi GPUs (data parallel)")
    parser.add_argument("opts", default=[], nargs=argparse.REMAINDER,
                        help="Modify config options using the command-line")
    args = parser.parse_args()
    return args


class AverageMeter(object):
    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val
        self.count += n
        self.avg = self.sum / self.count


def train(train_loader, model, criterion, optimizer, epoch, device):
    model.train()
    loss_monitor = AverageMeter()
    accuracy_monitor = AverageMeter()

    with tqdm(train_loader) as _tqdm:
        for x, y in _tqdm:
            x = x.to(device)
            y = y.to(device)

            # compute output
            outputs = model(x)

            # calc loss
            loss = criterion(outputs, y)
            cur_loss = loss.item()

            # calc accuracy
            _, predicted = outputs.max(1)
            correct_num = predicted.eq(y).sum().item()

            # measure accuracy and record loss
            sample_num = x.size(0)
            loss_monitor.update(cur_loss, sample_num)
            accuracy_monitor.update(correct_num, sample_num)

            # compute gradient and do SGD step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _tqdm.set_postfix(OrderedDict(stage="train", epoch=epoch, loss=loss_monitor.avg),
                              acc=accuracy_monitor.avg, correct=correct_num, sample_num=sample_num)

    return loss_monitor.avg, accuracy_monitor.avg


def validate(validate_loader, model, criterion, epoch, device):
    model.eval()
    loss_monitor = AverageMeter()
    accuracy_monitor = AverageMeter()
    preds = []
    gt = []

    with torch.no_grad():
        with tqdm(validate_loader) as _tqdm:
            for i, (x, y) in enumerate(_tqdm):
                x = x.to(device)
                y = y.to(device)

                # compute output
                outputs = model(x)
                preds.append(F.softmax(outputs, dim=-1).cpu().numpy())
                gt.append(y.cpu().numpy())

                # valid for validation, not used for test
                if criterion is not None:
                    # calc loss
                    loss = criterion(outputs, y)
                    cur_loss = loss.item()

                    # calc accuracy
                    _, predicted = outputs.max(1)
                    correct_num = predicted.eq(y).sum().item()

                    # measure accuracy and record loss
                    sample_num = x.size(0)
                    loss_monitor.update(cur_loss, sample_num)
                    accuracy_monitor.update(correct_num, sample_num)
                    _tqdm.set_postfix(OrderedDict(stage="val", epoch=epoch, loss=loss_monitor.avg),
                                      acc=accuracy_monitor.avg, correct=correct_num, sample_num=sample_num)

    preds = np.concatenate(preds, axis=0)
    gt = np.concatenate(gt, axis=0)
    ages = np.arange(0, 101)
    ave_preds = (preds * ages).sum(axis=-1)
    diff = ave_preds - gt
    mae = np.abs(diff).mean()

    return loss_monitor.avg, accuracy_monitor.avg, mae


# backup model checkpoint to drive to avoid loss of colab session restart
def model_to_drive(model):
    # is the instance running in colab
    # if 'google.colab' in str(get_ipython()):
    import os
    if "COLAB_GPU" in os.environ:
      import subprocess
      src = f"./checkpoint/{model}"
      dst = "/content/drive/MyDrive/age_esti_colab_session/"
      ret = subprocess.run(["cp", src, dst])
      if ret.returncode == 0:
          return True
    return False



def main():
    args = get_args()

    if args.opts:
        cfg.merge_from_list(args.opts)

    cfg.freeze()
    start_epoch = 0
    checkpoint_dir = Path(args.checkpoint)
    checkpoint_dir.mkdir(parents=True, exist_ok=True)

    # create model
    print("=> creating model '{}'".format(cfg.MODEL.ARCH))
    model = get_model(model_name=cfg.MODEL.ARCH)

    if cfg.TRAIN.OPT == "sgd":
        optimizer = torch.optim.SGD(model.parameters(), lr=cfg.TRAIN.LR,
                                    momentum=cfg.TRAIN.MOMENTUM,
                                    weight_decay=cfg.TRAIN.WEIGHT_DECAY)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=cfg.TRAIN.LR)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    # optionally resume from a checkpoint
    resume_path = args.resume

    if resume_path:
        if Path(resume_path).is_file():
            print("=> loading checkpoint '{}'".format(resume_path))
            checkpoint = torch.load(resume_path, map_location="cpu")
            start_epoch = checkpoint['epoch']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(resume_path, checkpoint['epoch']))
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        else:
            print("=> no checkpoint found at '{}'".format(resume_path))

    if args.multi_gpu:
        model = nn.DataParallel(model)

    if device == "cuda":
        cudnn.benchmark = True

    criterion = nn.CrossEntropyLoss().to(device)
    train_dataset = FaceDataset(args.data_dir, "train", img_size=cfg.MODEL.IMG_SIZE, augment=True,
                                age_stddev=cfg.TRAIN.AGE_STDDEV)
    train_loader = DataLoader(train_dataset, batch_size=cfg.TRAIN.BATCH_SIZE, shuffle=True,
                              num_workers=cfg.TRAIN.WORKERS, drop_last=True)

    val_dataset = FaceDataset(args.data_dir, "valid", img_size=cfg.MODEL.IMG_SIZE, augment=False)
    val_loader = DataLoader(val_dataset, batch_size=cfg.TEST.BATCH_SIZE, shuffle=False,
                            num_workers=cfg.TRAIN.WORKERS, drop_last=False)

    scheduler = StepLR(optimizer, step_size=cfg.TRAIN.LR_DECAY_STEP, gamma=cfg.TRAIN.LR_DECAY_RATE,
                       last_epoch=start_epoch - 1)
    best_val_mae = 10000.0
    train_writer = None

    if args.tensorboard is not None:
        opts_prefix = "_".join(args.opts)
        train_writer = SummaryWriter(log_dir=args.tensorboard + "/" + opts_prefix + "_train")
        val_writer = SummaryWriter(log_dir=args.tensorboard + "/" + opts_prefix + "_val")

    for epoch in range(start_epoch, cfg.TRAIN.EPOCHS):
        # train
        train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, device)

        # validate
        val_loss, val_acc, val_mae = validate(val_loader, model, criterion, epoch, device)

        if args.tensorboard is not None:
            train_writer.add_scalar("loss", train_loss, epoch)
            train_writer.add_scalar("acc", train_acc, epoch)
            val_writer.add_scalar("loss", val_loss, epoch)
            val_writer.add_scalar("acc", val_acc, epoch)
            val_writer.add_scalar("mae", val_mae, epoch)

        # checkpoint
        if val_mae < best_val_mae:
            print(f"=> [epoch {epoch:03d}] best val mae was improved from {best_val_mae:.3f} to {val_mae:.3f}")
            model_state_dict = model.module.state_dict() if args.multi_gpu else model.state_dict()
            torch.save(
                {
                    'epoch': epoch + 1,
                    'arch': cfg.MODEL.ARCH,
                    'state_dict': model_state_dict,
                    'optimizer_state_dict': optimizer.state_dict()
                },
                str(checkpoint_dir.joinpath("epoch{:03d}_{:.5f}_{:.4f}.pth".format(epoch, val_loss, val_mae)))
            )

            # copy to drive
            model_name = "epoch{:03d}_{:.5f}_{:.4f}.pth".format(epoch, val_loss, val_mae)
            ret = model_to_drive(model_name)
            if ret:
                print("model saved to drive")
            else:
                print("error occured copying model")

            best_val_mae = val_mae
        else:
            print(f"=> [epoch {epoch:03d}] best val mae was not improved from {best_val_mae:.3f} ({val_mae:.3f})")

        # adjust learning rate
        scheduler.step()

    print("=> training finished")
    print(f"additional opts: {args.opts}")
    print(f"best val mae: {best_val_mae:.3f}")


if __name__ == '__main__':
    main()


Writing train.py


load recent checkpoint into colab

in the current state of the cell,
this cell will have to constantly modified to reflect recent checkpoint

In [ ]:
!mkdir checkpoint   
# !cp /content/drive/MyDrive/age_esti_colab_session/new_after_101_mod/epoch074_0.02330_3.8380.pth ./checkpoint/

!cp /content/drive/MyDrive/age_esti_colab_session/epoch052_0.02836_7.3968.pth ./checkpoint/

# train runner

In [ ]:
# !python train.py --data_dir data_dir/appa-real-release --resume model_path --tensorboard tf_log101
# --resume checkpoint/epoch074_0.02330_3.8380.pth
# resume @ to continue
# epoch002_0.03012_8.0168.pth
!python train.py --data_dir data_dir/appa-real-release --tensorboard tf_log101 # --resume ./checkpoint/epoch052_0.02836_7.3968.pth

=> creating model 'se_resnext101_32x4d'
100% 124/124 [01:31<00:00,  1.36it/s, stage=train, epoch=0, loss=0.167, acc=0.0255, correct=0, sample_num=32]
100% 12/12 [00:23<00:00,  1.93s/it, stage=val, epoch=0, loss=0.0416, acc=0.028, correct=5, sample_num=92]
=> [epoch 000] best val mae was improved from 10000.000 to 11.970
model saved to drive
100% 124/124 [01:25<00:00,  1.46it/s, stage=train, epoch=1, loss=0.17, acc=0.0297, correct=0, sample_num=32]
100% 12/12 [00:14<00:00,  1.20s/it, stage=val, epoch=1, loss=0.0406, acc=0.03, correct=3, sample_num=92]
=> [epoch 001] best val mae was improved from 11.970 to 11.494
model saved to drive
100% 124/124 [01:25<00:00,  1.45it/s, stage=train, epoch=2, loss=0.161, acc=0.03, correct=0, sample_num=32]
100% 12/12 [00:14<00:00,  1.19s/it, stage=val, epoch=2, loss=0.041, acc=0.0207, correct=2, sample_num=92]
=> [epoch 002] best val mae was not improved from 11.494 (12.681)
100% 124/124 [01:25<00:00,  1.45it/s, stage=train, epoch=3, loss=0.163, acc=0.0

# visualize results

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir tf_log101/

<IPython.core.display.Javascript object>

# eval

In [ ]:
%%writefile test.py

import argparse
import better_exceptions
from pathlib import Path
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
from torch.utils.data import DataLoader
import pretrainedmodels
import pretrainedmodels.utils
from model import get_model
from dataset import FaceDataset
from defaults import _C as cfg
from train import validate


def get_args():
    model_names = sorted(name for name in pretrainedmodels.__dict__
                         if not name.startswith("__")
                         and name.islower()
                         and callable(pretrainedmodels.__dict__[name]))
    parser = argparse.ArgumentParser(description=f"available models: {model_names}",
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument("--data_dir", type=str, required=True, help="Data root directory")
    parser.add_argument("--resume", type=str, required=True, help="Model weight to be tested")
    parser.add_argument("opts", default=[], nargs=argparse.REMAINDER,
                        help="Modify config options using the command-line")
    args = parser.parse_args()
    return args


def main():
    args = get_args()

    if args.opts:
        cfg.merge_from_list(args.opts)

    cfg.freeze()

    # create model
    print("=> creating model '{}'".format(cfg.MODEL.ARCH))
    model = get_model(model_name=cfg.MODEL.ARCH, pretrained=None)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    # load checkpoint
    resume_path = args.resume

    if Path(resume_path).is_file():
        print("=> loading checkpoint '{}'".format(resume_path))
        checkpoint = torch.load(resume_path, map_location="cpu")
        model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}'".format(resume_path))
    else:
        raise ValueError("=> no checkpoint found at '{}'".format(resume_path))

    if device == "cuda":
        cudnn.benchmark = True

    test_dataset = FaceDataset(args.data_dir, "test", img_size=cfg.MODEL.IMG_SIZE, augment=False)
    test_loader = DataLoader(test_dataset, batch_size=cfg.TEST.BATCH_SIZE, shuffle=False,
                             num_workers=cfg.TRAIN.WORKERS, drop_last=False)

    print("=> start testing")
    _, _, test_mae = validate(test_loader, model, None, 0, device)
    print(f"test mae: {test_mae:.3f}")


if __name__ == '__main__':
    main()


Writing test.py


In [ ]:
!python test.py --data_dir data_dir/appa-real-release/ --resume checkpoint/epoch078_0.02325_3.8091.pth

=> creating model 'se_resnext101_32x4d'
Traceback (most recent call last):
  File "test.py", line 72, in <module>
    main()
  File "test.py", line 57, in main
    raise ValueError("=> no checkpoint found at '{}'".format(resume_path))
ValueError: => no checkpoint found at 'checkpoint/epoch078_0.02325_3.8091.pth'


# export logs

In [ ]:
!mv tf_log101 /content/drive/MyDrive/age_esti_colab_session/